## Тестирование стратегии на промежутке дней (год)
- запуск тестового алгоритма на любом временном промежутке
- возможность запуска разных комбинаций настроек и сравнения результатов

In [2]:

%reload_ext autoreload
%autoreload 2

from bot.vis import TaskProgress
from bot import TestAlgorithm
from app.config import RunConfig


from concurrent.futures import ThreadPoolExecutor, as_completed

TICKER = 'HNFG'

test_configs = [
    (RunConfig(
        ticker=TICKER,
        step_max_cnt=max_shares,
        step_base_cnt=base_shares,
        step_lots=2,

        majority_trade=True,
        pretest_period=pretest_period,
        pretest_type=pretest_type,

        threshold_buy_steps=threshold_b_n,
        threshold_sell_steps=threshold_s_n,
        stop_up_p=stop_up_p,
        stop_down_p=stop_down_p,

        step_size=step_size,
        step_set_orders_cnt=step_cnt,
    ))
    for max_shares in [3, 4]
    for base_shares in [0]
    for threshold_b_n in [0]
    for threshold_s_n in [0]
    for step_size in [1.4]
    for step_cnt in [3]
    # for pretest_period in range(2, 7)
    # for pretest_type in [ConfigDTO.PRETEST_PRE]
    for pretest_period in [13]
    for pretest_type in [RunConfig.PRETEST_RSI]
    for stop_up_p in [0]
    for stop_down_p in [0]
    # for max_shares in [4, 5]
    # for base_shares in [0]
    # for threshold_b_n in [0, max_shares+1]
    # for threshold_s_n in [0, max_shares+1]
    # for step_size in [0.8, 1, 1.2, 1.4]
    # for step_size_shift in [0, 0.25, 0.5]
    # for step_cnt in [2]
    # for pretest_period in [13]
    # for stop_up_p in [0]
    # for stop_down_p in [0]
]


def run_test(config: RunConfig):
    test_alg = TestAlgorithm(do_printing=False, config=config)
    return test_alg.test(
        last_test_date='2024-07-01',
        test_days_num=30,  # 365,
        shares_count=0,
    )

results = []
progress = TaskProgress(len(test_configs))

with ThreadPoolExecutor(max_workers=4) as executor:
    future_to_params = {executor.submit(run_test, config): config for config in test_configs}
    
    for future in as_completed(future_to_params):
        params = future_to_params[future]
        res = future.result()
        if res:
            results.append(res)
        progress.update_progress()

# Вывод результатов или их дальнейшая обработка
sorted_results = sorted(results, key=lambda x: float(x['profit_p']), reverse=True)

print()
for item in sorted_results:
    print(item)

ImportError: cannot import name 'TradingBot' from partially initialized module 'bot' (most likely due to a circular import) (/Users/a.sapozhkov/PycharmProjects/fin/bot/__init__.py)

Запуск без доработок нелинейного запуска
```
    return test_alg.test(
        last_test_date='2024-07-01',
        test_days_num=30,  # 365,
    )

{'profit': -1074.6, 'profit_p': -11.91, 'profit_p_avg': -0.4, 'config': HNFG+ 3/rsi13:0/3 x l2 x 1.4¤ , 'op': 1016}
{'profit': -1479.6, 'profit_p': -12.3, 'profit_p_avg': -0.41, 'config': HNFG+ 4/rsi13:0/3 x l2 x 1.4¤ , 'op': 1335}
```

запуск с доработками
```

```

RSI
{'profit': -87.4, 'profit_p': -3.11, 'config': RNFT+ 3/rsi13:0/3 x l2 x 1.4¤ , 'op': 181}
{'profit': -167.0, 'profit_p': -4.46, 'config': RNFT+ 4/rsi13:0/3 x l2 x 1.4¤ , 'op': 222}
{'profit': -149.1, 'profit_p': -5.31, 'config': RNFT+ 3/rsi13:0/3 x l2 x 1.2¤ , 'op': 190}
{'profit': -223.3, 'profit_p': -5.96, 'config': RNFT+ 4/rsi13:0/3 x l2 x 1.2¤ , 'op': 238}

если гулять по количеству дней
{'profit': 26.0, 'profit_p': 0.93, 'config': RNFT+ 3/rsi6:0/3 x l2 x 1.6¤ , 'op': 159}
{'profit': 26.0, 'profit_p': 0.93, 'config': RNFT+ 3/rsi7:0/3 x l2 x 1.6¤ , 'op': 159}
{'profit': 31.5, 'profit_p': 0.84, 'config': RNFT+ 4/rsi6:0/3 x l2 x 1.6¤ , 'op': 204}
{'profit': 31.5, 'profit_p': 0.84, 'config': RNFT+ 4/rsi7:0/3 x l2 x 1.6¤ , 'op': 204}
{'profit': 6.9, 'profit_p': 0.25, 'config': RNFT+ 3/rsi6:0/3 x l2 x 1.4¤ , 'op': 195}

PRE
{'profit': 87.1, 'profit_p': 3.1, 'config': RNFT+ 3/pre2:0/3 x l2 x 1.4¤ , 'op': 189}
{'profit': 87.1, 'profit_p': 3.1, 'config': RNFT+ 3/pre4:0/3 x l2 x 1.4¤ , 'op': 189}
{'profit': 87.1, 'profit_p': 3.1, 'config': RNFT+ 3/pre3:0/3 x l2 x 1.4¤ , 'op': 189}
{'profit': 87.1, 'profit_p': 3.1, 'config': RNFT+ 3/pre5:0/3 x l2 x 1.4¤ , 'op': 189}



#82 Динамическая подстройка параметров запуска
---

исходный вариант алгоритма
{'profit': 549.7, 'profit_p': 55.41, 'config': RNFT 4/pre13:0/3 x l2 x 1.2¤, |s0 b0| |u0.0 d0.0| maj+z+ , 'op_cnt': 2318}

на pre
{'profit': 253.8, 'profit_p': 51.17, 'config': RNFT 4/pre13:0/3 x l1 x 1.2¤, |s0 b0| |u0.0 d0.0| maj+z+ , 'op_cnt': 2318}

last_test_date='2024-05-01',
test_days_num=365,  # 365,
auto_conf_days_freq=7,
auto_conf_prev_days=7,
{'profit': 42.5, 'profit_p': 8.57, 'config': RNFT 4/0/3 x l1 x 1.2¤, |s0 b0| |u0.0 d0.0| maj+z+ , 'op_cnt': 2283}





И еще pretest_period подкрутил в 13 (было 20)
{'profit': 509.1, 'profit_p': '1.3', 'config': 4/pre13(3) x 1.2 rub, |s0 b5| maj+z+ }

Повторный выбор лучших параметров плюс новый алгоритм на продажу
{'profit': 369.6, 'profit_p': '0.94', 'config': 4/pre10(3) x 1.2 rub, |s0 b5| maj+z+ }

А вот так, если каждый день выходить из мажоритарной торговли
{'profit': 519.0, 'profit_p': '0.79', 'config': step 5/-5(2) x 1.4 rub, |s0 b6| pre- maj+, 'profit_avg': 2.07, 'pot_profit': 222.5, 'pot_profit_p': 2.33, 'days': 251, 'success_days': 139, 'success_p': 0.55, 'op_cnt': 4860, 'op_avg': 19.36}
на 365 дней - 'profit': 398.0, 'profit_p': '0.81', 'config': step 5/-5(2) x 1.4 rub

А с включенным режимом всегда -5 выдает вот так. но это без учета комиссии за непокрытые позиции
{'profit': 684.9, 'profit_p': '1.05', 'config': step 5/-5(2) x 1.4 rub, |s0 b6| pre- maj+, 'profit_avg': 2.73, 'pot_profit': 222.5, 'pot_profit_p': 3.08, 'days': 251, 'success_days': 144, 'success_p': 0.57, 'op_cnt': 3201, 'op_avg': 12.75}

на 5 шагов
{'profit': 437.4, 'profit_p': '0.67', 'config': step 5/5(5) x 1.4 rub, |s0 b6| pre10 maj+, 'profit_avg': 1.74, 'pot_profit': 222.5, 'pot_profit_p': 1.97, 'days': 251, 'success_days': 122, 'success_p': 0.49, 'op_cnt': 2127, 'op_avg': 8.47}


Все враньё. нашел ошибку в тестовых расчетах на 1 января - прирастало 660 руб. Реальные данных такие

с маж
{'profit': 399.0, 'profit_p': '0.61', 'config': step 5/5(2) x 1.4 rub, |s0 b6| pre10 maj+, 'profit_avg': 1.59, 'pot_profit': 222.5, 'pot_profit_p': 1.79, 'days': 251, 'success_days': 119, 'success_p': 0.47, 'op_cnt': 2221, 'op_avg': 8.85}

без маж
{'profit': 147.9, 'profit_p': '0.23', 'config': step 5/5(2) x 1.4 rub, |s0 b6| pre10 maj-, 'profit_avg': 0.59, 'pot_profit': 222.5, 'pot_profit_p': 0.66, 'days': 251, 'success_days': 108, 'success_p': 0.43, 'op_cnt': 1547, 'op_avg': 6.16}

без маж и без торговли в дни низкого RSI
{'profit': 149.6, 'profit_p': '0.23', 'config': step 5/5(2) x 1.4 rub, |s0 b6| pre10 maj-, 'profit_avg': 0.6, 'pot_profit': 222.5, 'pot_profit_p': 0.67, 'days': 251, 'success_days': 61, 'success_p': 0.24, 'op_cnt': 769, 'op_avg': 3.06}

без RSI
{'profit': -109.1, 'profit_p': '-0.17', 'config': step 5/5(2) x 1.4 rub, |s0 b6| pre- maj-, 'profit_avg': -0.43, 'pot_profit': 222.5, 'pot_profit_p': -0.49, 'days': 251, 'success_days': 112, 'success_p': 0.45, 'op_cnt': 1067, 'op_avg': 4.25}

-----

v8 первая версия обратной торговли
{'profit': 1079.5, 'profit_p': '1.65', 'config': step 5/5(2) x 1.4 rub, |s0 b6| pre+, 'profit_avg': 4.3, 'pot_profit': 222.5, 'pot_profit_p': 4.85, 'days': 251, 'success_days': 125, 'success_p': 0.5, 'op_cnt': 2223, 'op_avg': 8.86}

Если отключить вообще торговлю на низком RSI
'profit': 830.6, 'profit_p': '1.27'
{'profit': 830.6, 'profit_p': '1.27', 'config': step 5/5(2) x 1.4 rub, |s0 b6| pre+(10) maj-, 'profit_avg': 3.31, 'pot_profit': 222.5, 'pot_profit_p': 3.73, 'days': 251, 'success_days': 65, 'success_p': 0.26, 'op_cnt': 771, 'op_avg': 3.07}

Использование индекса относительной силы (RSI)
'profit': 828.7, 'profit_p': '1.27'
{'profit': 828.7, 'profit_p': '1.27', 'config': step 5/5(2) x 1.4 rub, |s0 b6|, 'profit_avg': 3.3, 'pot_profit': 222.5, 'pot_profit_p': 3.72, 'days': 251, 'success_days': 112, 'success_p': 0.45, 'op_cnt': 1551, 'op_avg': 6.18}

Подбор параметров 
'profit': 572.4, 'profit_p': '0.87'
{'profit': 572.4, 'profit_p': '0.87', 'profit_change_avg': 2.28, 'pot_profit': 222.5, 'pot_profit_p': 2.57, 'days': 251, 'success_days': 117, 'success_p': 0.47, 'sleep_trading': 60, 'max_shares': 5, 'base_shares': 5, 'threshold_buy_steps': 6, 'threshold_sell_steps': 0, 'step_size': 1.4, 'step_cnt': 2, 'operations_cnt': 1069, 'operations_avg': 4.26}

## v7

v7 с профитом
{'profit': 528.2, 'profit_p': '0.81', 'profit_change_avg': 2.1, 'pot_profit': 222.5, 'pot_profit_p': 2.37, 'days': 251, 'success_days': 117, 'success_p': 0.47, 'sleep_trading': 60, 'max_shares': 5, 'base_shares': 5, 'threshold_buy_steps': 5, 'threshold_sell_steps': 0, 'step_size': 1.4, 'step_cnt': 4, 'operations_cnt': 1049, 'operations_avg': 4.18}

переписал
'balance': 268.0, 'success_p': 0.52
{'balance': 268.0, 'profit_p': '0.19', 'balance_change_avg': 1.07, 'days': 251, 'success_days': 131, 'success_p': 0.52, 'sleep_trading': 60, 'max_shares': 8, 'base_shares': None, 'threshold_buy_steps': 4, 'threshold_sell_steps': 0, 'step_size': 1.2, 'step_cnt': 3, 'operations_cnt': 5310, 'operations_avg': 21.16, 'avg': 0.5, 'avg_': 0.0}

## v6 Первый запуск

покупка в начале, продажа в конце, фикс учитывающий изменение баланса
'balance': 114.7, 'success_p': 0.71, 

добавлена первичная покупка и продажа в конце дня
'balance': -347.9, 'success_p': 0.68

тест разных вариантов настроек
по проценту - 'balance': -26235.8, 'success_p': 0.73
по сумме - 'balance': -2329.3, 'success_p': 0.23,

подрезание заявок снизу
'balance': -16860.0, 'success_p': 0.62

```
год -54607.4 руб  'success_p': 0.46
{'balance': -54607.4, 'balance_change_avg': -217.56, 'days': 251, 'success_days': 115, 'success_p': 0.46, 'profit_steps': 5, 'candles_count': 4, 'sleep_trading': 300, 'take_profit_percent': 1.5, 'quit_on_balance_up_percent': 2, 'quit_on_balance_down_percent': 1, 'operations_cnt': 4859, 'operations_avg': 19.36, 'op_not_closed': 0, 'op_not_closed_avg': 0.0}
```

```
день +1.8 руб
{'balance': 1.8, 'balance_change_avg': 1.8, 'days': 1, 'success_days': 1, 'success_p': 1.0, 'profit_steps': 5, 'candles_count': 4, 'sleep_trading': 60, 'take_profit_percent': 1.5, 'quit_on_balance_up_percent': 2, 'quit_on_balance_down_percent': 1, 'operations_cnt': 12, 'operations_avg': 12.0, 'op_not_closed': 0, 'op_not_closed_avg': 0.0}
```



## V5 последние показатели

```
[{'balance': -170.1,
  'balance_change_avg': -0.68,
  'days': 251,
  'success_days': 71,
  'success_p': 0.28,
  'profit_steps': 5,
  'candles_count': 4,
  'sleep_trading': 60,
  'take_profit_percent': 1.5,
  'quit_on_balance_up_percent': 2,
  'quit_on_balance_down_percent': 1,
  'operations_cnt': 2758,
  'operations_avg': 10.99,
  'op_not_closed': 171,
  'op_not_closed_avg': 0.68}]
```

## V4 результаты

```
[{'balance': -327.3,
  'balance_change_avg': -1.3,
  'days': 251,
  'success_days': 58,
  'success_p': 0.23,
  'profit_steps': 5,
  'candles_count': 4,
  'sleep_trading': 60,
  'operations_cnt': 5020,
  'operations_avg': 20.0,
  'op_not_closed': 134,
  'op_not_closed_avg': 0.53}]
```

> {'balance': -193.7, 'balance_change_avg': -0.77, 'days': 251, 'success_days': 65, 'success_p': 0.26, 'profit_steps': 5, 'candles_count': 4, 'sleep_trading': 300, 'operations_cnt': 1390, 'operations_avg': 5.54, 'op_not_closed': 148, 'op_not_closed_avg': 0.59}
-> -120

> {'balance': -138.9, 'balance_change_avg': -0.55, 'days': 251, 'success_days': 60, 'success_p': 0.24, 'profit_steps': 5, 'candles_count': 4, 'sleep_trading': 600, 'operations_cnt': 780, 'operations_avg': 3.11, 'op_not_closed': 112, 'op_not_closed_avg': 0.45}
-> -54

> {'balance': -327.3, 'balance_change_avg': -1.3, 'days': 251, 'success_days': 58, 'success_p': 0.23, 'profit_steps': 5, 'candles_count': 4, 'sleep_trading': 60, 'operations_cnt': 5020, 'operations_avg': 20.0, 'op_not_closed': 134, 'op_not_closed_avg': 0.53}
-> -170

In [ ]:
# Поля, которые нужно исключить
fields = [
    'config',
    # 'days',
    # 'sleep_trading',
    # 'step_cnt',
#     'profit_avg',
#     'profit',
#     'pot_profit_p',
]

# Отфильтровываем данные, исключая указанные поля
filtered_data = [
    {key: item[key] for key in item if key not in fields}
    for item in sorted_results
]


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# data = sorted_results
data = filtered_data

df = pd.DataFrame(data)
correlation_matrix = df.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.show()